In [1]:
#=========================================================================================================
#문장쌍 STS dataset 이 적은 경우(label이 적은 경우) 예시
#
# -1단계: 적은 STS dataset(gold sts dataset) 에 대해 Cross-Encoder로 BERT 훈련 시킴
# -2.1단계: 기존 잘 훈련된 S-BERT(예: distiluse-base-multilingual-cased-v2)를 이용해, gold sts 문장들에 대해 유사도 측정해서, 
#           한 문장에 대해 K수만큼 유사한 문장들을 조합하여 문장 쌍을 만듬
# -2.2단계: 1E단계에서 훈련된 BERT 로 2.1단계에서 만든 문장쌍들에 대해 점수를 매김->이를 silver sts dataset이라고 함
# -3단계: gold sts dataset + silver sts dataset 을 훈련 데이터로 하여 Bi-Encoder 훈련 시킴
#=========================================================================================================

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

import math
from sentence_transformers import models, losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('../../')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()
seed_everything(111)

logfilepath:../../log/bwdataset_2022-04-25.log
logfilepath:../../log/qnadataset_2022-04-25.log
logfilepath:../../log/s-bert_2022-04-25.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
# 모델들 정의
max_seq_len = 128
train_batch_size = 32
num_epochs = 10
top_k = 3             # 훈련데이터에서 몇개의 유사도 문장을 뽑아낼지 정하는 값

#======================================================================================================
# cross-encoder 모델 정의
ce_model_path = "../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/"
ce_model = CrossEncoder(ce_model_path, num_labels=1)
#======================================================================================================

#======================================================================================================
# 훈련 데이터들의 유사도 문장들를 구할 bi-encoder 모델 정의
# => distiluse-base-multilingual-cased-v2 를 이용
bi_encoder_path = "../../../model/sbert/teacher/distiluse-base-multilingual-cased-v2/"

word_embedding_encoder = models.Transformer(bi_encoder_path, max_seq_length=max_seq_len)

pooling_encoder = models.Pooling(word_embedding_encoder.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

bi_encoder = SentenceTransformer(modules=[word_embedding_encoder, pooling_encoder])
#======================================================================================================

print(ce_model)
print(bi_encoder)

Some weights of the model checkpoint at ../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [3]:
# 1단계: 적은 STS dataset(gold sts dataset) 에 대해 Cross-Encoder로 BERT 훈련 시킴
# => 여기서는 5,000개만 있는 korsts/tune_train.tsv 말뭉치를 가지고 함.

train_file = '../../../korpora/korsts/tune_train.tsv'
eval_file = '../../../korpora/korsts/tune_dev.tsv'
count = 0

# 훈련 골드 데이터 불러옴
train_gold_samples = []
with open(train_file, "rt", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        text_a, text_b, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
        if count < 5:
            print(f"{text_a}, {text_b}, {score}")
                
        train_gold_samples.append(InputExample(texts= [text_a,text_a], label=score))
        count += 1
      
# 평가 데이터 불러옴
count = 0
dev_samples = []
with open(eval_file, 'rt', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        text_a, text_b, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
        if count < 5:
            print(f"{text_a}, {text_b}, {score}")
            
        dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
        
# 훈련 골드 데이터로더, 평가 데이터 로더 생성
train_dataloader = DataLoader(train_gold_samples, shuffle=True, batch_size=train_batch_size)
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

print(train_gold_samples[0:2])
print(dev_samples[0:2])

비행기가 이륙하고 있다., 비행기가 이륙하고 있다., 1.0
한 남자가 큰 플루트를 연주하고 있다., 남자가 플루트를 연주하고 있다., 0.76
한 남자가 피자에 치즈를 뿌려놓고 있다., 한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다., 0.76
세 남자가 체스를 하고 있다., 두 남자가 체스를 하고 있다., 0.52
한 남자가 첼로를 연주하고 있다., 자리에 앉은 남자가 첼로를 연주하고 있다., 0.85
안전모를 가진 한 남자가 춤을 추고 있다., 안전모를 쓴 한 남자가 춤을 추고 있다., 1.0
어린아이가 말을 타고 있다., 아이가 말을 타고 있다., 0.95
한 남자가 뱀에게 쥐를 먹이고 있다., 남자가 뱀에게 쥐를 먹이고 있다., 1.0
한 여성이 기타를 연주하고 있다., 한 남자가 기타를 치고 있다., 0.48
한 여성이 플루트를 연주하고 있다., 남자가 플루트를 연주하고 있다., 0.55
[<sentence_transformers.readers.InputExample.InputExample object at 0x7faa084f1850>, <sentence_transformers.readers.InputExample.InputExample object at 0x7faa084dce20>]
[<sentence_transformers.readers.InputExample.InputExample object at 0x7fa77bb91220>, <sentence_transformers.readers.InputExample.InputExample object at 0x7fa77bb913a0>]


In [4]:
# 1단계: 적은 STS dataset(gold sts dataset) 에 대해 Cross-Encoder로 BERT 훈련 시킴

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

ce_odel_save_path = 'output/crossencoder-sts-train-'+datetime.now().strftime("%Y-%m-%d-%H:%M")

# Train the model
ce_model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=ce_odel_save_path)

2022-04-25 11:33:27,950 - s-bert - INFO - Warmup-steps: 180


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

In [5]:
# -2.1단계: 기존 잘 훈련된 S-BERT(예: distiluse-base-multilingual-cased-v2)를 이용해, gold sts 문장들에 대해 유사도 측정해서, 
#           한 문장에 대해 K수만큼 유사한 문장들을 조합하여 문장 쌍을 만듬
# ** 굳이 gold sts 문장에셔 siver sts 를 생성하지 않고, 다른 말뭉치에서 생성해도 될듯 싶음.

silver_train_data = []
sentences = set()
for sample in train_gold_samples:
    sentences.update(sample.texts)
   
sentences = list(sentences)
print(len(sentences))
print(sentences[0:5])

sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)}
duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in train_gold_samples)

embeddings = bi_encoder.encode(sentences, batch_size=train_batch_size, convert_to_tensor=True)

for idx in tqdm(range(len(sentences)), unit="docs"):
    sentence_embedding = embeddings[idx]
    cos_scores = util.pytorch_cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx and (iid, idx) not in duplicates:
            silver_train_data.append((sentences[idx], sentences[iid]))
            duplicates.add((idx, iid))
            
# -2.2단계: 1E단계에서 훈련된 BERT 로 2.1단계에서 만든 문장쌍들에 대해 점수를 매김->이를 silver sts dataset이라고 함
cs_new_model = CrossEncoder(ce_odel_save_path)
silver_scores = cs_new_model.predict(silver_train_data)
    
print(len(silver_train_data))
print(silver_train_data[0:5])
print(silver_scores[0:5])

5384
['나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '인도 조건은 이탈리아 당국이 기소할 충분한 증거를 제공할 수 있는 서류 위조에 대한 단일 혐의에 대해서만 튀니지인이 기소될 수 있도록 규정하고 있다.', '원숭이가 나무에 매달려 있다.', '러시아와 나토 회원국들은 올 가을 유럽 조약(cfe )에서 재래식 세력에 대한 회의를 개최할 것이다.', '그러나 합동 정보 위원회는 관여하지 않았다는 것은 분명하다.']


  0%|          | 0/5384 [00:00<?, ?docs/s]

26920
[('나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '나토 : 아프가니스탄에서 사망한 2명의 국제군'), ('나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '아프가니스탄에서 4명의 나토군이 사망했다.'), ('나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '나토 : 아프가니스탄 남부에서 5명의 미국인이 사망'), ('나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '4명의 민간인, 3명의 나토 병사가 아프가니스탄에서 사망했다'), ('나토 : 아프가니스탄 저항 세력으로부터 구조된 4명의 구호 요원', '아프가니스탄에서 살해된 나토 병사 한 명')]
[0.4914771  0.6120426  0.40192905 0.61064124 0.60000867]


In [6]:
# silver_data 를 .tsv 파일로 저장 해둠.
silver_data_file = 'output/silver_data-'+datetime.now().strftime("%Y-%m-%d-%H:%M")+'.tsv'
with open(silver_data_file, 'w', encoding='utf-8') as f:
    for data, score in tqdm(zip(silver_train_data, silver_scores)):
        f.write(data[0]+'\t')
        f.write(data[1]+'\t')
        f.write(str(score) + '\n')    

In [11]:
# 3단계: gold sts dataset + silver sts dataset 을 훈련 데이터로 하여 Bi-Encoder 훈련 시킴

#======================================================================================================
# bi-encoder 모델 정의

# word_embedding 모델은 앞에서 할 cross-encoder 모델 경로로 지정함 
word_embedding_model = models.Transformer(ce_model_path, max_seq_length=max_seq_len)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

bi_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
#======================================================================================================

train_silver_samples = list(InputExample(texts=[data[0], data[1]], label=score) for \
    data, score in zip(silver_train_data, silver_scores))

train_dataset = SentencesDataset(train_gold_samples + train_silver_samples, bi_model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=bi_model)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training.
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
#logging.info("Warmup-steps: {}".format(warmup_steps))

model_save_path = 'output/sbert-sts-train-'+datetime.now().strftime("%Y-%m-%d_%H-%M")

# Train the bi-encoder model
bi_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )


Some weights of the model checkpoint at ../../../model/bert/bmc-fpt-bong_corpus_mecab-0424/batch:32-ep:4-lr:0.000030000-4m25d-8:27/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../../../model/bert/bmc-fpt-bong_

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1021 [00:00<?, ?it/s]